In [1]:
import pandas as pd 


ibge = pd.read_csv("/home/lisamenezes/Searches/pipeline-dados-PDM/data/silver/ibge_goiania.csv")

tse = pd.read_csv("/home/lisamenezes/Searches/pipeline-dados-PDM/data/silver/tse_2012_bairros.csv")

In [3]:
tse.head()

,Unnamed: 0,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,QT_VOTOS,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO
0,0,2012,GOIANIA,1,1,2,1,VEREADOR,ADAILA LUZ PAES LEME,1,['COLEGIO AGOSTINIANO NOSSA SENHORA DE FATIMA'...,"['AV K, 108 - UNIDADE I', 'AV E QD. B-11', 'RU...","['SETOR AEROPORTO', 'JARDIM GOIAS', 'SETOR OES..."
1,1,2012,GOIANIA,1,1,2,1,VEREADOR,ADRIANA RODRIGUES PINHEIRO DIOGO,1,['COLEGIO AGOSTINIANO NOSSA SENHORA DE FATIMA'...,"['AV K, 108 - UNIDADE I', 'AV E QD. B-11', 'RU...","['SETOR AEROPORTO', 'JARDIM GOIAS', 'SETOR OES..."
2,2,2012,GOIANIA,1,1,2,1,VEREADOR,ADRIANO RODRIGO LINO MALTEZ,3,['COLEGIO AGOSTINIANO NOSSA SENHORA DE FATIMA'...,"['AV K, 108 - UNIDADE I', 'AV E QD. B-11', 'RU...","['SETOR AEROPORTO', 'JARDIM GOIAS', 'SETOR OES..."
3,3,2012,GOIANIA,1,1,2,1,VEREADOR,AGNALDO BURJACK EVANGELISTA,1,['COLEGIO AGOSTINIANO NOSSA SENHORA DE FATIMA'...,"['AV K, 108 - UNIDADE I', 'AV E QD. B-11', 'RU...","['SETOR AEROPORTO', 'JARDIM GOIAS', 'SETOR OES..."
4,4,2012,GOIANIA,1,1,2,1,VEREADOR,AILMA MARIA DE OLIVEIRA,3,['COLEGIO AGOSTINIANO NOSSA SENHORA DE FATIMA'...,"['AV K, 108 - UNIDADE I', 'AV E QD. B-11', 'RU...","['SETOR AEROPORTO', 'JARDIM GOIAS', 'SETOR OES..."


In [5]:
tse.shape

(285225, 13)

### Ultimas transformações em IBGE para agregação gold

Vamos ter que entender essa tabela como uma amostra, porque eu filtrei teoricamente pela cidade, então, se o censo passar por todas as casas, no minimo deveria ter 1 milhão de linhas esse dataset

In [4]:
# Converter as colunas numéricas para o tipo float
ibge["V005"] = ibge["V005"].str.replace(",", ".").astype(float)
ibge["V009"] = ibge["V009"].str.replace(",", ".").astype(float)
ibge.head(2)

,Unnamed: 0,Nome_do_subdistrito,Nome_do_bairro,Situacao_setor,V001,V002,V005,V007,V009,V011
0,3239,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,276.0,642.0,2432.47,"2543,03",1588.10,"2084,38"
1,3240,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,225.0,491.0,2198.14,"2389,29",1512.69,"2008,52"


In [6]:
# De salarios deixei apenas o que tenho certeza
# Acho que esses dados representam o setor cencitário, mas queremos so valores por setor

ibge = ibge[['Nome_do_subdistrito', 'Situacao_setor', 'V005', 'V009']]

# Converter as colunas numéricas para o tipo float
ibge["V005"] = pd.to_numeric(ibge["V005"], errors="coerce")
ibge["V009"] = pd.to_numeric(ibge["V009"], errors="coerce")


# Agregar os dados por Nome_do_subdistrito
ibge_agrupado = ibge.groupby("Nome_do_subdistrito").agg({
    "Situacao_setor": lambda x: x.mode()[0],  # Pega o valor mais comum
    "V005": "mean",  # Média dos valores
    "V009": "mean",  # Média dos valores
}).reset_index()

# Ver o resultado
ibge_agrupado

,Nome_do_subdistrito,Situacao_setor,V005,V009
0,U.T.P. AEROPORTO,1,3679.232000,2220.962000
1,U.T.P. AEROPORTO INTERNACIONAL SANTA GENOVEVA,1,3435.710000,1542.860000
2,U.T.P. AEROVIARIOS,1,1243.974737,806.283158
3,U.T.P. ALTO DA GLORIA E REDENCAO,1,3067.470400,2045.876400
4,U.T.P. AUTODROMO OU PARQUE LOZANDES,1,8360.048000,4542.501000
...,...,...,...,...
59,U.T.P. VILA PEDROSO,1,880.640800,577.509600
60,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1,1199.879286,750.282143
61,U.T.P. VILA RIZZO,1,874.826000,597.820667
62,U.T.P. ZONA RURAL,1,860.635952,536.335714


In [7]:
ibge_agrupado['Nome_do_subdistrito'].unique()

array(['U.T.P. AEROPORTO',
       'U.T.P. AEROPORTO INTERNACIONAL SANTA GENOVEVA',
       'U.T.P. AEROVIARIOS', 'U.T.P. ALTO DA GLORIA E REDENCAO',
       'U.T.P. AUTODROMO OU PARQUE LOZANDES', 'U.T.P. BAIRRO FELIZ',
       'U.T.P. BALIZA-ITAIPU', 'U.T.P. BUENO', 'U.T.P. CAMPINAS',
       'U.T.P. CAMPUS UNIVERSITARIO/CONJUNTO ITATIAIA',
       'U.T.P. CANDIDA DE MORAIS/MARIA DILCE', 'U.T.P. CAPUAVA',
       'U.T.P. CARAVELAS', 'U.T.P. CEASA/ALDEIA DO VALE',
       'U.T.P. CELINA PARK/RECREIO DOS FUNCIONARIOS', 'U.T.P. CENTRAL',
       'U.T.P. CHACARAS SAO JOAQUIM', 'U.T.P. CIDADE JARDIM',
       'U.T.P. COIMBRA', 'U.T.P. CRIMEIA LESTE', 'U.T.P. FINSOCIAL',
       'U.T.P. GOIANIA II', 'U.T.P. JAO', 'U.T.P. JARDIM AMERICA',
       'U.T.P. JARDIM ATLANTICO',
       'U.T.P. JARDIM BALNEARIO MEIA PONTE/MANSOES GOIANAS',
       'U.T.P. JARDIM EUROPA', 'U.T.P. JARDIM GOIAS',
       'U.T.P. JARDIM GUANABARA', 'U.T.P. JARDIM NOVO MUNDO',
       'U.T.P. JARDIM PETROPOLIS',
       'U.T.P. JARDIM 

#### Agora, vamos queremos juntas ibge com tse

In [8]:
print((tse['BAIRRO'].isnull()).sum())

12192


In [9]:
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 1.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 946.3 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 1.4 MB/s eta 0:00:00a 0:00:01


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Função para encontrar melhor match por similaridade de texto
def match_tfidf(bairros, ibge_subdistritos):
    vectorizer = TfidfVectorizer().fit(ibge_subdistritos)
    
    # Vetorizar os bairros e subdistritos
    bairros_vetorizados = vectorizer.transform(bairros)
    subdistritos_vetorizados = vectorizer.transform(ibge_subdistritos)
    
    # Calcular a similaridade de coseno
    similaridades = cosine_similarity(bairros_vetorizados, subdistritos_vetorizados)
    melhores_indices = similaridades.argmax(axis=1)
    
    # Retornar o melhor match
    return [ibge_subdistritos[idx] for idx in melhores_indices]

# Preparar os bairros em formato de texto único para TF-IDF
tse["BAIRRO"] = tse["BAIRRO"].apply(
    lambda x: x if isinstance(x, list) else ['Bairro não identificado']
)
# Preparar os bairros em formato de texto único para TF-IDF
tse["bairro_str"] = tse["BAIRRO"].apply(lambda x: " ".join(x))


# Realizar o match e adicionar as informações de ibge
tse_matches = []
for _, row in tse.iterrows():
    bairros = [row["bairro_str"]]
    matches = match_tfidf(bairros, ibge["Nome_do_subdistrito"])
    
    # Pegar o primeiro match
    subdistrito_info = ibge_agrupado[ibge_agrupado["Nome_do_subdistrito"] == matches[0]].iloc[0]
    combined_row = row.to_dict()
    combined_row.update({
        "Nome_do_subdistrito": subdistrito_info["Nome_do_subdistrito"],
        "Situacao_setor": subdistrito_info["Situacao_setor"],
        "V005": subdistrito_info["V005"],
        "V009": subdistrito_info["V009"]
    })
    tse_matches.append(combined_row)

# Criar DataFrame unificado
tse_unificado = pd.DataFrame(tse_matches)

# Ver o resultado
print(tse_unificado)

KeyboardInterrupt: 